In [17]:
import numpy as np
from core import DynamicalSystem
from plotting import evolution_plot, phase_portrait, evolution_dot_plot
from rich.console import Console
from rich.traceback import install
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import HoverTool, ColumnDataSource
from tqdm import tqdm

console = Console()
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x122586cf0>>

In [ ]:
def system(t, state_vector, params):
    x, y, phi1, phi2 = state_vector
    omega, gamma, kappa, eps, Omega, L1, L2, L3 = params
    eta = -kappa/2
    beta = np.tan(gamma)
    nu = omega + eta*beta
    epsc = eps/np.cos(gamma)
    Wt = Wt

    # forcing terms
    Fx = L1*np.cos(Wt) + L2*np.cos(3*Wt) + L3*np.cos(5*Wt)
    Fy = L1*np.sin(Wt) + L2*np.sin(3*Wt) + L3*np.sin(5*Wt)

    f1 = L1*np.sin(1*Wt-phi1-gamma)
    f2 = L2*np.sin(3*Wt-phi1-gamma)
    f3 = L3*np.sin(5*Wt-phi1-gamma)

    s1 = L1*np.sin(1*Wt-phi2-gamma)
    s2 = L2*np.sin(3*Wt-phi2-gamma)
    s3 = L3*np.sin(5*Wt-phi2-gamma)

    d1 = (omega-1*Omega)/kappa
    d2 = (omega-3*Omega)/kappa
    d3 = (omega-5*Omega)/kappa

    s11 = L1*L1*np.cos(d1)*(np.sin(d1)+np.sin(2*(2*Wt-phi2)-d1))
    s12 = L1*L2*np.cos(d2)*(np.sin(-2*Wt))


    # cartesian equations
    xdot = eta*x - nu*y - eta*(x - beta*y)*(x**2 + y**2) + eps*Fx
    ydot = eta*y + nu*x - eta*(y + beta*x)*(x**2 + y**2) + eps*Fy

    # first-order phase equation
    phi_dot_1 = omega + epsc*(f1+f2+f3)

    return np.array([xdot, ydot, phi_dot_1], np.float64)

In [19]:
def frequency(t, state_vector, state_vector_dot):
    x, y, _ = state_vector
    xdot, ydot, phi_dot = state_vector_dot
    freq = (ydot*x - xdot*y)/(x**2 + y**2)
    return np.array([freq, phi_dot])

In [52]:
p = {
    'omega':    1.0,
    'gamma':    0.1,
    'kappa':    -2.0,
    'eps':      0.05,
    'Omega':    0.4,
    'L1':       1.00,
    'L2':       0.10,
    'L3':       0.01
}

# Set initial state and time span
u0 = {'x': 1.0, 'y': 0.0, 'phi': 0.0}

In [53]:
eps = p['eps']/np.cos(p['gamma'])
h = 0.5*(eps*p['L1']/p['kappa'])**2
mean_w = p['omega']*(1+h)/(3+h)
wing = p['eps']*p['L2']/(3+h)
left = mean_w - wing
right = mean_w + wing
print(left, right)

0.3317369781725719 0.33506996082507784


In [54]:
phs = DynamicalSystem(system, t0=0, x0=u0, parameters=p)

### Synchronization domain check

In [56]:
#freqs = phs.evaluate(frequency,50_000,100)

In [57]:
plt = figure(title="Time evolution", 
               x_axis_label="time",
               y_axis_label="variables",
               width=1200,
               height=600)
output_notebook()
plt.line(phs.t_sol[::1], phs.xdot_sol[2][::1],
            legend_label="phi",
            line_width=2, color='#344966')
#p.hspan(y=[phs.xdot_sol[2].mean()], line_color='#FF6666', line_width=2)
plt.hspan(y=[freqs[0]], line_color='#55D6BE', line_width=2)
plt.hspan(y=[freqs[1]], line_color='#E6AACE', line_width=2)
    
plt.add_tools(HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")]))
plt.legend.location = 'top_left'
show(plt)

Loading BokehJS ...

In [58]:
Omega_range = np.linspace(0.30, 0.36, 600)
run_time = 10_000
tr_time = 100

freqs = phs.run_parameter(frequency, 'Omega', Omega_range, 
                          t_range=run_time, tr=tr_time)

freq_xy = freqs[:,0]
freq_f1 = freqs[:,1]

In [51]:
from bokeh.models import Span
plt = figure(title="Synchronization domain", 
               x_axis_label="Omega",
               y_axis_label="freq/Omega",
               width=1200,
               height=600)
output_notebook()
plt.line(Omega_range, freq_xy/Omega_range,
            legend_label="Full XY",
            line_width=2, color='#344966')
plt.line(Omega_range, freq_f1/Omega_range,
            legend_label="First-order",
            line_width=2, color='#FF6666')

w, eps, kappa = p['omega'], p['eps'], p['kappa']
L1, L2, gamma = p['L1'], p['L2'], p['gamma']

# shift = 0.0695 # for eps = 0.3, gamma = 0.1
# shift = 0.125 # for eps = 0.4, gamma = 0.1
# shift = 0.123 # for eps = 0.4, gamma = 0.0

h = 0.5*(eps*L1/(kappa*np.cos(gamma)))**2

Omega_min_3 = (w - eps*(L2)/np.cos(gamma))/3
Omega_max_3 = (w + eps*(L2)/np.cos(gamma))/3
Omega_min_3s = (w*(1+h) - eps*L2/np.cos(gamma))/(3+h)
Omega_max_3s = (w*(1+h) + eps*L2/np.cos(gamma))/(3+h)

vline1 = Span(location=Omega_min_3, dimension='height', 
              line_color='red', line_width=2)
vline2 = Span(location=Omega_max_3, dimension='height', 
              line_color='red', line_width=2)

vline4 = Span(location=Omega_min_3, dimension='height', 
              line_color='grey', line_width=2)
vline5 = Span(location=Omega_max_3, dimension='height', 
              line_color='grey', line_width=2)

plt.add_layout(vline1)    
plt.add_layout(vline2)    
plt.add_layout(vline4)    
plt.add_layout(vline5)    

plt.add_tools(HoverTool(tooltips=[("Omega", "@x"), ("Ratio", "@y")]))
plt.legend.location = 'top_right'
show(plt)

Loading BokehJS ...

In [40]:
h

np.float64(0.0202013409284499)